In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import requests
import json
from datetime import datetime 

In [17]:
url =  "https://apis.11st.co.kr/pui/v2/page?pageId=APCCATEGORY&dispCtgr2No=1149914&benefit=MBSHP_PRD&blckSn=7933&pageMode=END"

In [18]:
def extract_product_data() :
    response = requests.get(url)

    products = []

    if response.status_code == 200 :
        data = response.json()

        # Loop through all blocks in 'blockList'
        for block in data['data'][0]['blockList']:
            # Check if the block contains products (based on its type)
            if block['type'] == 'Acme_PC_ProductBox_Category_Col4_Small':
                # Extract products from this block
                for item in block['list']:
                    # Ensure the required fields exist before extraction
                    if 'prdNm' in item:

                        # Extract promotionFlags text if available
                        promotion_flags = item.get('promotionFlags', [])
                        # Join multiple promotion texts if present
                        promotion_texts = ', '.join(flag.get('text', '') for flag in promotion_flags)
                        product_info = {
                            'Image URL': item.get('imageUrl', 'N/A'),
                            'Product Name': item.get('prdNm', 'N/A'),
                            'Original Price': item.get('sellPrice', 'N/A'),
                            'Discounted Price': item.get('finalDscPrice', 'N/A'),
                            'Discount Rate': item.get('discountRate', 'N/A') + item.get('discountRateUnitText', ''),
                            'Link': item.get('linkUrl', 'N/A'),
                            'Review Count': item.get('review', {}).get('count', 'N/A'),
                            'Review Score': item.get('review', {}).get('point', 'N/A'),
                            'Is Sold Out': item.get('isSoldOut', 'N/A'),
                            'Promotion Texts': promotion_texts  # Include promotion texts here
                        }
                        products.append(product_info)

            # Print the extracted product information
        for product in products:
            print(product)

        return products

    else :
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        return products

        

In [19]:
products = extract_product_data()

AttributeError: 'list' object has no attribute 'get'

In [15]:
# Directory to save the JSON file
json_directory = 'json'
os.makedirs(json_directory, exist_ok=True)  # Create the directory if it doesn't exist

# Save the product list as a JSON file with the current date
date_str = datetime.now().strftime('%Y-%m-%d')
file_name = f'products_{date_str}.json'
file_path = os.path.join(json_directory, file_name)  # Save the file in the 'json' directory

with open(file_path, 'w', encoding='utf-8') as file:
    json.dump(products, file, ensure_ascii=False, indent=4)

print(f"Product data saved: {file_path}")

Product data saved: json/products_2024-09-16.json
